In [80]:
import os
import glob
import re
import pandas as pd

In [78]:
def get_well_file(date, barcode, well=None):
    data_dir = get_data_dir(date, barcode)
    ### example file style
    f1 = barcode+".result."+well+"[test].csv"
    #f2 = barcode+".result."+well+"[test].csv"
    files = os.listdir(data_dir)
    if f1 in files:
        well_file = os.path.join(data_dir, f1)
    else:
        print("well csv not found!")
    return(well_file)


## input date: e.g. '2021-10-15'
## output: list of barcodes e.g. '211015_combo_173'
def get_barcodes(date):
    date_formatted = date_format_switch(date)
    main_dir = get_data_dir(date)
    dirs = [ x for x in os.listdir(main_dir) if os.path.isdir( os.path.join(main_dir, x) )]
    ### match date at the start of the sub-directory
    dirs_barcodes = [ x for x in dirs if bool(re.match(date_formatted+"_combo", x)) ]
    return( dirs_barcodes )

## input date: e.g. '2021-02-19'
## output: e.g. '210219'
def date_format_switch(date):
    new_str = date[2:4] + date[5:7] + date[8:10]
    return(new_str)

def get_data_dir(date, barcode=None, base_dir = "/mnt/y/lsp-analysis/LINCS-combinations/"):
    ### note: using unix folder conventions -- would need to re-write for Windows
    if barcode == None:
        plate_dir = ''
    else:
        plate_dir = barcode
    folder_dict = {
        '2020-11-17':'rep3',
        '2021-02-19':'rep4',
        '2021-02-26':'rep5',
        '2021-03-02':'rep6',
        '2021-04-06':'rep7',
        '2021-04-23':'rep8',
        '2021-05-18':'rep9',
        '2021-05-21':'rep10',
        '2021-06-11':'rep11',
        '2021-07-27':'2_rep1/210727-combo-rep1',
        '2021-07-30':'2_rep2/210730_combo_rep2',
        '2021-08-06':'2_rep3/210806_combo_rep3',
        '2021-10-05':'redo_rep1_and_2',
        '2021-10-15':'redo_rep1_and_2/redo_rep2',
        '2021-10-29':'redo_rep3'
    }
    local_dir = folder_dict[date]
    full_dir = os.path.join(base_dir, local_dir, plate_dir)
    return(full_dir)

In [25]:
get_data_dir('2021-02-19')
get_data_dir('2021-10-15', "211015_combo_176")



'/mnt/y/lsp-analysis/LINCS-combinations/redo_rep1_and_2/redo_rep2/211015_combo_176'

In [52]:
date_format_switch("2021-02-19")

'210219'

In [62]:
re.search("234", "abdsf234")

<re.Match object; span=(5, 8), match='234'>

In [74]:
get_barcodes('2021-10-15')

['211015_combo_173',
 '211015_combo_171',
 '211015_combo_169',
 '211015_combo_176',
 '211015_combo_175',
 '211015_combo_174',
 '211015_combo_168',
 '211015_combo_172',
 '211015_combo_170',
 '211015_combo_167']

In [81]:
test = get_well_file('2021-10-15', '211015_combo_173', 'C05')

In [82]:
pd.read_csv(test)

,Unnamed: 0,Plate ID,Well Name,Site ID,Cell ID,MEASUREMENT SET ID,Cell Count (DDD-bckgrnd),Cell: ObjectID (DDD-bckgrnd),Cell: EdUbackground (DDD-bckgrnd),Cell: LDRbackground (DDD-bckgrnd),Cell: EdUrawINT (DDD-bckgrnd),Cell: LDRrawINT (DDD-bckgrnd),Cell: HoechstINT (DDD-bckgrnd),ldrint
0,8984,16167,C05,1,16167_4209579_8461147_1,46348,NaN,1.0,2174.395264,955.744202,2324.078857,1555.210571,2373.631592,599.466369
1,8985,16167,C05,1,16167_4209579_8461147_10,46348,NaN,10.0,1894.406738,170.257462,1854.822266,170.177780,1671.722168,-0.079682
2,8986,16167,C05,1,16167_4209579_8461147_100,46348,NaN,100.0,1914.668701,189.137497,1968.922119,191.155838,2065.415527,2.018341
3,8987,16167,C05,1,16167_4209579_8461147_101,46348,NaN,101.0,2608.377441,184.874176,3612.977783,175.488892,2081.899902,-9.385284
4,8988,16167,C05,1,16167_4209579_8461147_102,46348,NaN,102.0,2024.284668,194.730774,2202.040039,194.453339,2248.186768,-0.277435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4162,13146,16167,C05,4,16167_4209582_8461150_995,46348,NaN,995.0,1991.727295,226.715912,2156.500000,256.868408,4155.315918,30.152496
4163,13147,16167,C05,4,16167_4209582_8461150_996,46348,NaN,996.0,1927.220581,324.661774,1914.180054,357.260010,2407.840088,32.598236
4164,13148,16167,C05,4,16167_4209582_8461150_997,46348,NaN,997.0,1966.873169,224.440292,1982.114990,224.504425,2922.247803,0.064133
4165,13149,16167,C05,4,16167_4209582_8461150_998,46348,NaN,998.0,1867.525024,205.193756,1986.285767,209.775513,3400.520508,4.581757
